In [1]:
import tensorflow as tf

In [2]:
from utils_20191230 import *

In [3]:
tf.logging.set_verbosity(tf.logging.INFO)

#### ETRI BERT Config
```python
bert_config = {
  "attention_probs_dropout_prob": 0.1, 
  "directionality": "bidi", 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 768, 
  "initializer_range": 0.02, 
  "intermediate_size": 3072, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 12, 
  "num_hidden_layers": 12, 
  "pooler_fc_size": 768, 
  "pooler_num_attention_heads": 12, 
  "pooler_num_fc_layers": 3, 
  "pooler_size_per_head": 128, 
  "pooler_type": "first_token_transform", 
  "type_vocab_size": 2, 
  "vocab_size": 30349
}
```

In [4]:
# load ETRI Bert Config
path = '../KorBERT/2_bert_download_002_bert_morp_tensorflow/002_bert_morp_tensorflow/'
FLAGS.bert_config_file = path + 'bert_config.json'

In [5]:
bert_config = BertConfig.from_json_file(FLAGS.bert_config_file)
# bert_config = BertConfig.from_dict(bert_config) # 위의 dictionary를 메모리에 올려서 
                                                 # 다음 메서드로 호출하는 것도 가능하다.

In [6]:
if FLAGS.max_seq_length > bert_config.max_position_embeddings:
    raise ValueError(
        "Cannot use sequence length %d because the BERT model "
        "was only trained up to sequence length %d" %
        (FLAGS.max_seq_length, bert_config.max_position_embeddings))
else:
    print('     (FLAGS) MAX_SEQ_LENGTH           :', FLAGS.max_seq_length)
    print('(BERTConfig) MAX_POSITION_EMBEDDINGS  :', bert_config.max_position_embeddings)

     (FLAGS) MAX_SEQ_LENGTH           : 128
(BERTConfig) MAX_POSITION_EMBEDDINGS  : 512


In [7]:
# do not use tpu
tpu_cluster_resolver = None
if FLAGS.use_tpu and FLAGS.tpu_name:
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
        FLAGS.tpu_name, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project)
else:
    print('Do not use TPU')

Do not use TPU


In [8]:
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps, # 1000
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop, # 1000
        num_shards=FLAGS.num_tpu_cores, # 8
        per_host_input_for_training=is_per_host)
)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [9]:
# tensorflow gpu 사용 가능한지 체크
tf.test.is_gpu_available()

True

In [10]:
# 한국어 vocab 사전을 등록
FLAGS.vocab_file = path + 'vocab.korean_morp.list' 

간략한 파일 준비

In [11]:
import pandas as pd

dacon_path = '../dacon문자스미싱/filedown (2)/'
df_train = pd.read_csv(dacon_path + 'train.csv')
df_test = pd.read_csv(dacon_path + 'public_test.csv')

In [12]:
from sklearn.model_selection import train_test_split

df_train = df_train.set_index('id')
df_test = df_test.set_index('id')

X_train, X_valid, y_train, y_valid = train_test_split(
                 df_train[[col for col in df_train.columns if col != 'smishing']], 
                 df_train['smishing'],
                 random_state=42, test_size=.2,
                 stratify=df_train['smishing'])
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((236756, 2), (59189, 2), (236756,), (59189,))

In [13]:
df_train = pd.concat((X_train, y_train), axis=1)
df_valid = pd.concat((X_valid, y_valid), axis=1)

In [14]:
# sample 100개씩 뽑아서 미리 test
df_train.sample(100).to_csv(dacon_path + 'train_100.tsv', sep='\t')
df_valid.sample(100).to_csv(dacon_path + 'dev_100.tsv', sep='\t')
df_test.sample(100).to_csv(dacon_path + 'test_100.tsv', sep='\t')

In [15]:
# DataProcessor 제작
class SmishingProcessor(DataProcessor):

    def get_train_examples(self, data_dir, filename='train.tsv'):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, filename)), "train")

    def get_dev_examples(self, data_dir, filename='dev.tsv'):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, filename)), "dev")

    def get_test_examples(self, data_dir, filename='test.tsv'):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, filename)), "test")

    def get_labels(self):
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = convert_to_unicode(line[2])
            if set_type == "test":
                label = "0"
            else:
                label = convert_to_unicode(line[-1])
            examples.append(
                InputExample(guid=guid, text_a=text_a, label=label))
        return examples

In [16]:
processor = SmishingProcessor()
label_list = processor.get_labels()

# get train samples
train_examples = processor.get_train_examples(dacon_path, 'train_100.tsv')
num_train_steps = int(
    len(train_examples) / FLAGS.train_batch_size * FLAGS.num_train_epochs)
num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)

In [22]:
# record ETRI model weights
FLAGS.init_checkpoint = path + 'model.ckpt'

In [24]:
model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=len(label_list),             # 2
    init_checkpoint=FLAGS.init_checkpoint,  # None
    learning_rate=FLAGS.learning_rate,      # 5e-05
    num_train_steps=num_train_steps,        # 22195
    num_warmup_steps=num_warmup_steps,      # 2219
    use_tpu=FLAGS.use_tpu,                  # False
    use_one_hot_embeddings=FLAGS.use_tpu)   # False

In [25]:
# If TPU is not available, this will fall back to normal Estimator on CPU
# or GPU
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,                        # False
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,      # 32
    eval_batch_size=FLAGS.eval_batch_size,        # 8
    predict_batch_size=FLAGS.predict_batch_size   # 8
)

INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\jinma\\AppData\\Local\\Temp\\tmp95ip6j47', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002196B1B3A20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, nu

In [26]:
FLAGS.output_dir = './output_dir/smishing/'

In [27]:
tf.gfile.MakeDirs(FLAGS.output_dir)

Embedding
- TTA 표준 형태소 태그셋(TTAK.KO-11.0010/R1)에 맞는 형태소 분석기를 사용해야 함.

<table class="table table-striped table-bordered" style="width:800px;">
  <thead>
    <tr>
      <th style="width:200px" align="center">대분류</td>
      <th style="width:200px" align="center">중분류</td>
      <th style="width:400px" align="center">대분류</td>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td rowspan="5" align="center">(1) 체언</td>
      <td rowspan="3" align="center">명사</td>
      <td align="center">일반명사(NNG)</td>
    </tr>
    <tr>
      <td align="center">고유명사(NNP)</td>
    </tr>
    <tr>
      <td align="center">의존명사(NNB)</td>
    </tr>
    <tr>
      <td align="center">대명사(NP)</td>
      <td align="center">대명사(NP)</td>
    </tr>
    <tr>
      <td align="center">수사(NR)</td>
      <td align="center">수사(NR)</td>
    </tr>
    <tr>
      <td rowspan="5" align="center">(2) 용언</td>
      <td align="center">동사(VV)</td>
      <td align="center">동사(VV)</td>
    </tr>
    <tr>
      <td align="center">형용사(VA)</td>
      <td align="center">형용사(VA)</td>
    </tr>
    <tr>
      <td align="center">보조용언(VX)</td>
      <td align="center">보조용언(VX)</td>
    </tr>
    <tr>
      <td rowspan="2" align="center">지정사(VC)</td>
      <td align="center">긍정지정사(VCP)</td>
    </tr>
    <tr>
      <td align="center">부정지정사(VCN)</td>
    </tr>
    <tr>
      <td rowspan="5" align="center">(3) 수식언</td>
      <td rowspan="3" align="center">관형사(MM)</td>
      <td align="center">성상 관형사(MMA)</td>
    </tr>
    <tr>
      <td align="center">지시 관형사(MMD)</td>
    </tr>
    <tr>
      <td align="center">수 관형사(MMN)</td>
    </tr>
    <tr>
      <td rowspan="2" align="center">부사(MA)</td>
      <td align="center">일반부사(MAG)</td>        
    </tr>
    <tr>
      <td align="center">접속부사(MAJ)</td>
    </tr>
    <tr>
      <td align="center">(4) 독립언</td>
      <td align="center">감탄사(IC)</td>
      <td align="center">감탄사(IC)</td>
    </tr>
    <tr>
      <td rowspan="9" align="center">(5) 관계언</td>
      <td rowspan="7" align="center">격조사(JK)</td>
      <td align="center">주격조사(JKS)</td>
    </tr>
    <tr>
      <td align="center">보격조사(JKC)</td>
    </tr>
    <tr>
      <td align="center">관형격조사(JKG)</td>
    </tr>
    <tr>
      <td align="center">목적격조사(JKO)</td>
    </tr>
    <tr>
      <td align="center">부사격조사(JKB)</td>
    </tr>
    <tr>
      <td align="center">호격조사(JKV)</td>
    </tr>
    <tr>
      <td align="center">인용격조사(JKQ)</td>
    </tr>
    <tr>
      <td align="center">보조사(JX)</td>
      <td align="center">보조사(JX)</td>
    </tr>
    <tr>
      <td align="center">접속조사(JC)</td>
      <td align="center">접속조사(JC)</td>
    </tr>
    <tr>
      <td rowspan="10" align="center">(6) 의존형태</td>
      <td rowspan="5" align="center">어미(EM)</td>
      <td align="center">선어말어미(EP)</td>
    </tr>
    <tr>
      <td align="center">종결어미(EF)</td>
    </tr>
    <tr>
      <td align="center">연결어미(EC)</td>
    </tr>
    <tr>
      <td align="center">명사형전성어미(ETN)</td>
    </tr>
    <tr>
      <td align="center">관형형전성어미(ETM)</td>
    </tr>
    <tr>
      <td align="center">접두사(XP)</td>
      <td align="center">체언접두사(XPN)</td>
    </tr>
    <tr>
      <td rowspan="3" align="center">접미사(XS)</td>
      <td align="center">명사파생접미사(XSN)</td>
    </tr>
    <tr>
      <td align="center">동사파생접미사(XSV)</td>
    </tr>
    <tr>
      <td align="center">형용사파생접미사(XSA)</td>
    </tr>
    <tr>
      <td align="center">어근(XR)</td>
      <td align="center">어근(XR)</td>
    </tr>
    <tr>
      <td rowspan="10" align="center">(7) 기초</td>
      <td rowspan="6" align="center">일반기호(ST)</td>
      <td align="center">마침표, 물음표, 느낌표(SF)</td>
    </tr>
    <tr>
      <td align="center">쉼표, 가운뎃점, 콜론, 빗금(SP)</td>
    </tr>  
    <tr>
      <td align="center">따옴표, 괄호표, 줄표(SS)</td>
    </tr>
    <tr>
      <td align="center">줄임표(SE)</td>
    </tr>
    <tr>
      <td align="center">붙임표(물결)(SO)</td>
    </tr>
    <tr>
      <td align="center">기타 기호(SW)</td>
    </tr>
    <tr>
      <td align="center">외국어(SL)</td>
      <td align="center">외국어(SL)</td>
    </tr>
    <tr>
      <td align="center">한자(SH)</td>
      <td align="center">한자(SH)</td>
    </tr>
    <tr>
      <td align="center">숫자(SN)</td>
      <td align="center">숫자(SN)</td>
    </tr>
    <tr>
      <td align="center">분석불능범주(NA)</td>
      <td align="center">분석불능범주(NA)</td>
    </tr>
  </tdoby>
</table>

In [38]:
# TTA 가이드라인
# http://aiopen.etri.re.kr/data/001.형태소분석_가이드라인.pdf
tta_guide = {
    '체언': {
        '명사': ('NN',
               {'일반명사': 'NNG',
                '고유명사': 'NNP',
                '의존명사': 'NNB'}),
        '대명사': ('NP',{'대명사': 'NP'}),
        '수사': ('NR', {'수사': 'NR'})
    },
    '용언': {
        '동사': ('VV', {'동사': 'VV'}),
        '형용사': ('VA', {'형용사': 'VA'}),
        '보조용언': ('VX', {'보조용언': 'VX'}),
        '지정사': ('VC', 
                {'긍정지정사': 'VCP',
                 '부정지정사': 'VCN'})
    },
    '수식언': {
        '관형사': ('MM', 
                {'성상 관형사': 'MMA',
                 '지시 관형사': 'MMD',
                 '수 관형사': 'MMN'}),
        '부사': ('MA', 
               {'일반부사': 'MAG',
                '접속부사': 'MAJ'})
    },
    '독립언': {
        '감탄사': ('IC', {'감탄사': 'IC'})
    },
    '관계언': {
        '격조사': ('JK', 
                {'주격조사': 'JKS',
                 '보격조사': 'JKC',
                 '관형격조사': 'JKG',
                 '목적격조사': 'JKO',
                 '부사격조사': 'JKB',
                 '호격조사': 'JKV',
                 '인용격조사': 'JKQ'}),
        '보조사': ('JX', {'보조사': 'JK'}),
        '접속조사': ('JC', {'접속조사': 'JC'})
    },
    '의존형태': {
        '어미': ('EM', 
               {'선어말어미': 'EP',
                '종결어미': 'EF',
                '연결어미': 'EC',
                '명사형전성어미': 'ETN',
                '관형형전성어미': 'ETM'}),
        '접두사': ('XP', {'체언접두사': 'XPN'}),
        '접미사': ('XS', 
                {'명사파생접미사': 'XSN',
                 '동사파생접미사': 'XSV',
                 '형용사파생접미사': 'XSA'}),
        '어근': ('XR', {'어근': 'XR'})
    },
    '기호': {
        '일반기호': ('ST', 
                 {'마침표, 물음표, 느낌표': 'SF',
                  '쉼표, 가운뎃점, 콜론, 빗금': 'SP',
                  '따옴표, 괄호표, 줄표': 'SS',
                  '줄임표': 'SE',
                  '붙임표(물결)': 'SO',
                  '기타 기호': 'SW'}),
        '외국어': ('SL', {'외국어': 'SL'}),
        '한자': ('SH', {'한자': 'SH'}),
        '숫자': ('SN', {'숫자': 'SN'}),
        '분석불능범주': ('NA', {'분석불능범주': 'NA'})
    }
}

In [39]:
tta_guide

{'체언': {'명사': ('NN', {'일반명사': 'NNG', '고유명사': 'NNP', '의존명사': 'NNB'}),
  '대명사': ('NP', {'대명사': 'NP'}),
  '수사': ('NR', {'수사': 'NR'})},
 '용언': {'동사': ('VV', {'동사': 'VV'}),
  '형용사': ('VA', {'형용사': 'VA'}),
  '보조용언': ('VX', {'보조용언': 'VX'}),
  '지정사': ('VC', {'긍정지정사': 'VCP', '부정지정사': 'VCN'})},
 '수식언': {'관형사': ('MM', {'성상 관형사': 'MMA', '지시 관형사': 'MMD', '수 관형사': 'MMN'}),
  '부사': ('MA', {'일반부사': 'MAG', '접속부사': 'MAJ'})},
 '독립언': {'감탄사': ('IC', {'감탄사': 'IC'})},
 '관계언': {'격조사': ('JK',
   {'주격조사': 'JKS',
    '보격조사': 'JKC',
    '관형격조사': 'JKG',
    '목적격조사': 'JKO',
    '부사격조사': 'JKB',
    '호격조사': 'JKV',
    '인용격조사': 'JKQ'}),
  '보조사': ('JX', {'보조사': 'JK'}),
  '접속조사': ('JC', {'접속조사': 'JC'})},
 '의존형태': {'어미': ('EM',
   {'선어말어미': 'EP',
    '종결어미': 'EF',
    '연결어미': 'EC',
    '명사형전성어미': 'ETN',
    '관형형전성어미': 'ETM'}),
  '접두사': ('XP', {'체언접두사': 'XPN'}),
  '접미사': ('XS', {'명사파생접미사': 'XSN', '동사파생접미사': 'XSV', '형용사파생접미사': 'XSA'}),
  '어근': ('XR', {'어근': 'XR'})},
 '기호': {'일반기호': ('ST',
   {'마침표, 물음표, 느낌표': 'SF',
    '쉼

In [63]:
pos_set = []
for VALUE in tta_guide.values():
    for VALUE2 in VALUE.values():
        pos_set.append(VALUE2[0])
        for VALUE3 in VALUE2[1].values():
            pos_set.append(VALUE3)
pos_set = list(set(pos_set))

In [65]:
from konlpy.tag import Komoran

komoran = Komoran()

In [67]:
[i[1] for i in komoran.pos(train_examples[0].text_a) if i[1] not in pos_set]

[]

Komoran 형태소 분석기로 분석 실시

In [81]:
for exam in train_examples:
    exam.text_a = ' '.join(
        [i[0] + '/' + i[1] 
         for i in komoran.pos(exam.text_a)])

In [82]:
train_examples[0].text_a

'XXX/SL 지점/NNP 을/JKO 거래/NNG 하/XSV 아/EC 주/VX 시/EP 어서/EC 대단히/MAG 감사/NNG 하/XSV ㅂ니다/EF ./SF 내/NNP 점/NNB 하/NNP XXX/NNP 고객/NNG 님/XSN 고객/NNG 만족도/NNG 설문/NNP 조사/NNP 전화/NNG 받/VV 으시/EP 면/EC 매우/MAG 동의/NNG 하/XSV ㄴ다라고/EC 우수/NNP 직원/NNP 추천/NNG 해주시/NNP 이/VCP 고/EC 사은품/NNG 받/VV 아/EC 가/VX 시/EP 어요/EC ../SE 더욱더/MAG 친절히/MAG 모시/VV 겠/EP 습니다/EC XXX/SL 은행/NNP XXX/NNP 올림/NNP'

In [243]:
train_file = os.path.join(FLAGS.output_dir, 'train.tf_record')

In [106]:
# fn: file_based_convert_examples_to_features

## Arguments
examples = train_examples
label_list = label_list
max_seq_length = FLAGS.max_seq_length
# tokenizer = FullTokenizer() # 예시로 tokenizing을 어떻게 하는지 전부 기록
output_file = train_file

In [107]:
writer = tf.python_io.TFRecordWriter(output_file)

In [108]:
ex_index = 5
example = examples[ex_index]

In [109]:
# fn: convert_single_example
isinstance(example, PaddingInputExample)

False

In [110]:
label_map = {}
for (i, label) in enumerate(label_list):
    label_map[label] = i
label_map

{'0': 0, '1': 1}

In [111]:
# class: FullTokenizer
path = '../KorBERT/2_bert_download_002_bert_morp_tensorflow/002_bert_morp_tensorflow/'
FLAGS.vocab_file = path + 'vocab.korean_morp.list' 
vocab_file = FLAGS.vocab_file
do_lower_case = FLAGS.do_lower_case
text = example.text_a
text

'XXX 고객님항상 XXX은행 모란역 지점을 이용해 주시는 고객님께 감사의 마음을 전합니다. 혹시 업무와 관련해 궁금한 점이 있으시면 이 번호로 연락주시기바랍니다. 성심껏 도와드리겠습니다. 또 혹시 고객만족도 조사 전화를 받으시면 매우 동의한다 로 칭찬해 주세요 조금은 쌀쌀한 10월의 첫주입니다.환절기  감기조심하시고 따듯한 차와 함께 건강한 한주 보내시기 바랍니다.XXX은행모란역XXX올림'

In [112]:
vocab = load_vocab(vocab_file)
inv_vocab = {v:k for k, v in vocab.items()}

In [113]:
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
openapi_key = ''
requestJson = { "access_key": openapi_key, "argument": { "text": text, "analysis_code": "morp" } }

In [114]:
requestJson

{'access_key': '0da70af6-e163-44b5-8d5d-f217bebb5765',
 'argument': {'text': 'XXX 고객님항상 XXX은행 모란역 지점을 이용해 주시는 고객님께 감사의 마음을 전합니다. 혹시 업무와 관련해 궁금한 점이 있으시면 이 번호로 연락주시기바랍니다. 성심껏 도와드리겠습니다. 또 혹시 고객만족도 조사 전화를 받으시면 매우 동의한다 로 칭찬해 주세요 조금은 쌀쌀한 10월의 첫주입니다.환절기  감기조심하시고 따듯한 차와 함께 건강한 한주 보내시기 바랍니다.XXX은행모란역XXX올림',
  'analysis_code': 'morp'}}

In [115]:
import urllib3

In [116]:
http = urllib3.PoolManager()

In [117]:
response = http.request( "POST", openApiURL, headers={"Content-Type": "application/json; charset=UTF-8"}, body=json.dumps(requestJson))

In [118]:
json_data = json.loads(response.data.decode('utf-8'))
json_result = json_data["result"]

In [119]:
json_result

0

In [120]:
json_data = json.loads(response.data.decode('utf-8'))
json_return_obj = json_data["return_object"]
return_result = ""
json_sentence = json_return_obj["sentence"]
for json_morp in json_sentence:                        
    for morp in json_morp["morp"]:
        return_result = return_result+str(morp["lemma"])+"/"+str(morp["type"])+" "

In [121]:
return_result

'XXX/SL 고객/NNG 님/XSN 항상/MAG XXX/SL 은행/NNG 모란/NNG 역/NNG 지점/NNG 을/JKO 이용/NNG 하/XSV 어/EC 주/VX 시/EP 는/ETM 고객/NNG 님/XSN 께/JKB 감사/NNG 의/JKG 마음/NNG 을/JKO 전하/VV ㅂ니다/EF ./SF 혹시/MAG 업무/NNG 와/JKB 관련/NNG 하/XSV 어/EC 궁금하/VA ㄴ/ETM 점/NNG 이/JKS 있/VA 으시/EP 면/EC 이/MM 번호/NNG 로/JKB 연락/NNG 주/VV 시/EP 기/ETN 바라/VV ㅂ니다/EF ./SF 성심껏/MAG 돕/VV 아/EC 드리/VX 겠/EP 습니다/EF ./SF 또/MAG 혹시/MAG 고객/NNG 만족/NNG 도/NNG 조사/NNG 전화/NNG 를/JKO 받/VV 으시/EP 면/EC 매우/MAG 동의/NNG 하/XSV ㄴ다/EF 로/JKB 칭찬/NNG 하/XSV 어/EC 주/VX 시/EP 어요/EF 조금/NNG 은/JX 쌀쌀하/VA ㄴ/ETM 10/SN 월/NNB 의/JKG 첫주/NNG 이/VCP ㅂ니다/EF ./SF 환절/NNG 기/XSN 감기/NNG 조심/NNG 하/XSV 시/EP 고/EC 따듯하/VA ㄴ/ETM 차/NNG 와/JC 함께/MAG 건강/NNG 하/XSA ㄴ/ETM 한/MM 주/NNB 보내/VV 시/EP 기/ETN 바라/VV ㅂ니다/EF ./SF XXX/SL 은행/NNG 모란/NNG 역/NNG XXX/SL 올림/NNG '

In [124]:
' '.join([i[0] + '/' + i[1] for i in komoran.pos(text)])

'XXX/SL 고객/NNG 님/NNG 항상/MAG XXX/SL 은행/NNP 모란역/NNP 지점/NNG 을/JKO 이용/NNG 하/XSV 아/EC 주시/NNP 는/JX 고객/NNG 님/XSN 께/JKB 감사/NNG 의/JKG 마음/NNG 을/JKO 전하/VV ㅂ니다/EF ./SF 혹시/MAG 업무/NNG 와/JC 관련/NNG 하/XSV 아/EC 궁금/XR 하/XSA ㄴ/ETM 점/NNB 이/JKS 있/VX 으시/EP 면/EC 이/MM 번호/NNG 로/JKB 연락/NNG 주/NNG 시기/NNG 바라/VV ㅂ니다/EF ./SF 성심껏/MAG 돕/VV 아/EC 드리/VX 겠/EP 습니다/EF ./SF 또/MAJ 혹시/MAG 고객/NNP 만족/NNP 도/JX 조사/NNG 전화/NNG 를/JKO 받/VV 으시/EP 면/EC 매우/MAG 동의/NNG 하/XSV ㄴ다/EC 로/NNG 칭찬/NNG 하/XSV 아/EC 주/VX 시/EP 어요/EC 조금/NNG 은/JX 쌀쌀/XR 하/XSA ㄴ/ETM 10월/NNP 의/JKG 첫/MM 주/NNB 이/VCP ㅂ니다/EF ./SF 환절기/NNG 감기/NNP 조심/NNG 하/XSV 시/EP 고/EC 따듯/XR 하/XSA ㄴ/ETM 차/NNG 와/JC 함께/MAG 건강/NNG 하/XSV ㄴ/ETM 한/MM 주/NNP 보내/VV 시/EP 기/ETN 바라/VV ㅂ니다/EF ./SF XXX/SL 은행/NNP 모란역/NNP XXX/NNP 올림/NNP'

In [125]:
def do_lang ( openapi_key, text ) :
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
	 
    requestJson = { "access_key": openapi_key, "argument": { "text": text, "analysis_code": "morp" } }
	 
    http = urllib3.PoolManager()
    response = http.request( "POST", openApiURL, headers={"Content-Type": "application/json; charset=UTF-8"}, body=json.dumps(requestJson))
    
    json_data = json.loads(response.data.decode('utf-8'))
    json_result = json_data["result"]
    
    if json_result == -1:
        json_reason = json_data["reason"]
        if "Invalid Access Key" in json_reason:
            logger.info(json_reason)
            logger.info("Please check the openapi access key.")
            sys.exit()
        return "openapi error - " + json_reason      
    else:
        json_data = json.loads(response.data.decode('utf-8'))
    
        json_return_obj = json_data["return_object"]
        
        return_result = ""
        json_sentence = json_return_obj["sentence"]
        for json_morp in json_sentence:                        
            for morp in json_morp["morp"]:
                return_result = return_result+str(morp["lemma"])+"/"+str(morp["type"])+" "

        return return_result

In [127]:
return_result

'XXX/SL 고객/NNG 님/XSN 항상/MAG XXX/SL 은행/NNG 모란/NNG 역/NNG 지점/NNG 을/JKO 이용/NNG 하/XSV 어/EC 주/VX 시/EP 는/ETM 고객/NNG 님/XSN 께/JKB 감사/NNG 의/JKG 마음/NNG 을/JKO 전하/VV ㅂ니다/EF ./SF 혹시/MAG 업무/NNG 와/JKB 관련/NNG 하/XSV 어/EC 궁금하/VA ㄴ/ETM 점/NNG 이/JKS 있/VA 으시/EP 면/EC 이/MM 번호/NNG 로/JKB 연락/NNG 주/VV 시/EP 기/ETN 바라/VV ㅂ니다/EF ./SF 성심껏/MAG 돕/VV 아/EC 드리/VX 겠/EP 습니다/EF ./SF 또/MAG 혹시/MAG 고객/NNG 만족/NNG 도/NNG 조사/NNG 전화/NNG 를/JKO 받/VV 으시/EP 면/EC 매우/MAG 동의/NNG 하/XSV ㄴ다/EF 로/JKB 칭찬/NNG 하/XSV 어/EC 주/VX 시/EP 어요/EF 조금/NNG 은/JX 쌀쌀하/VA ㄴ/ETM 10/SN 월/NNB 의/JKG 첫주/NNG 이/VCP ㅂ니다/EF ./SF 환절/NNG 기/XSN 감기/NNG 조심/NNG 하/XSV 시/EP 고/EC 따듯하/VA ㄴ/ETM 차/NNG 와/JC 함께/MAG 건강/NNG 하/XSA ㄴ/ETM 한/MM 주/NNB 보내/VV 시/EP 기/ETN 바라/VV ㅂ니다/EF ./SF XXX/SL 은행/NNG 모란/NNG 역/NNG XXX/SL 올림/NNG '

In [130]:
ids_to_tokens = collections.OrderedDict(
    [(ids, tok) for tok, ids in vocab.items()])

In [132]:
never_split=("[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]")

In [133]:
do_lower_case

True

In [135]:
vocab_file

'../KorBERT/2_bert_download_002_bert_morp_tensorflow/002_bert_morp_tensorflow/vocab.korean_morp.list'

In [138]:
os.path.isdir(vocab_file)

False

In [139]:
from pathlib import Path

from torch.hub import _get_torch_home

torch_cache_home = _get_torch_home()
    
default_cache_path = os.path.join(torch_cache_home, "transformers")

PYTORCH_PRETRAINED_BERT_CACHE = Path(
    os.getenv("PYTORCH_TRANSFORMERS_CACHE", os.getenv("PYTORCH_PRETRAINED_BERT_CACHE", default_cache_path))
)

In [141]:
TRANSFORMERS_CACHE = PYTORCH_PRETRAINED_BERT_CACHE

In [142]:
isinstance(vocab_file, Path)

False

In [144]:
if isinstance(TRANSFORMERS_CACHE, Path):
    cache_dir = str(TRANSFORMERS_CACHE)
cache_dir

'C:\\Users\\jinma\\.cache\\torch\\transformers'

In [148]:
from urllib.parse import urlparse

def is_remote_url(url_or_filename):
    parsed = urlparse(url_or_filename)
    return parsed.scheme in ('http', 'https', 's3')

In [272]:
tokens = []
tokens.append('[CLS]')
tokens

['[CLS]']

In [149]:
is_remote_url(vocab_file)

False

In [150]:
os.path.exists(vocab_file)

True

In [151]:
def cached_path(
    url_or_filename, cache_dir=None, force_download=False, proxies=None, resume_download=False, user_agent=None
):
    if cache_dir is None:
        cache_dir = TRANSFORMERS_CACHE
    if isinstance(url_or_filename, Path):
        url_or_filename = str(url_or_filename)
    if isinstance(cache_dir, Path):
        cache_dir = str(cache_dir)

    if is_remote_url(url_or_filename):
        # URL, so get it from the cache (downloading if necessary)
        return get_from_cache(
            url_or_filename,
            cache_dir=cache_dir,
            force_download=force_download,
            proxies=proxies,
            resume_download=resume_download,
            user_agent=user_agent,
        )
    elif os.path.exists(url_or_filename):
        # File, and it exists.
        return url_or_filename
    elif urlparse(url_or_filename).scheme == "":
        # File, but it doesn't exist.
        raise EnvironmentError("file {} not found".format(url_or_filename))
    else:
        # Something unknown
        raise ValueError("unable to parse {} as a URL or as a local path".format(url_or_filename))

In [152]:
cached_path(vocab_file) # vocab file 반환

'../KorBERT/2_bert_download_002_bert_morp_tensorflow/002_bert_morp_tensorflow/vocab.korean_morp.list'

In [94]:
def _clean_text(text):
    output = [] # char을 저장할 list 생성
    for char in text:
        # 텍스트에서 Char 단위로 출력
        cp = ord(char)
        if cp == 0 or cp == 0xfffd or _is_control(char):
            # \x00이거나 �이거나 unicode cat.이 C로 시작할 경우
            # (개행문자 제외) output에 추가하지 않는다.
            continue
        if _is_whitespace(char):
            # 공백일 경우 " "으로 output에 추가
            output.append(" ")
        else:
            # 이 외의 경우 전부 output에 추가
            output.append(char)
    # cleaning 작업을 거친 Text를 후처리하여 반환
    return "".join(output)

# char 단위 함수들
def _is_whitespace(char):
    if char == " " or char == '\t' or char == '\n' or char == '\r':
        # 개행문자이거나 띄어쓰기면 True 반환
        return True
    cat = unicodedata.category(char)
    if cat == 'Zs':
        # unicode category가 Space Seperator면 True 반환
        return True
    # 이 외의 경우 전부 False 반환
    return False

def _is_control(char):
    if char == "\t" or char == "\n" or char == "\r":
        # 개행문자이면 False 반환
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        # unicode category가
        # Cc(Control) 
        # Cf(format)
        # Co(Private Use, is 0)
        # Cs(Surrrogate, is 0)일 경우, True 반환
        return True
    # 이 외의 경우 전부 False 반환
    return False

def _is_punctuation(char):
    # 한국어 형태소 분석기이기 때문에 공백과 같은지 여부만 반환
    return char == ' '

In [157]:
def whitespace_tokenize(text):
	"""Runs basic whitespace cleaning and splitting on a peice of text."""
	text = text.strip()
	if not text:
		return []
	tokens = text.split()
	return tokens

In [95]:
def print_c(text, is_print):
    if is_print:
        print(text)
    else:
        print(end='')

In [248]:
# do_lower_case = False
do_lower_case = True

In [258]:
print('곧/NNG')
print(unicodedata.normalize("NFD", '곧/NNG'))
print('곧/NNG' == unicodedata.normalize("NFD", '곧/NNG'))
print('곧/NNG' == unicodedata.normalize("NFC",
                    unicodedata.normalize("NFD", '곧/NNG')))
print(unicodedata.normalize("NFC", '곧/NNG') == '곧/NNG')

곧/NNG
곧/NNG
False
True
True


In [264]:
'{:>30}'.format('--|> Exists in vocab_file.')

'    --|> Exists in vocab_file.'

In [270]:
# FullTokenizer.tokenize(); End2End Tokenizer
text = copy.copy(return_result) # text 초기화
print('************** START TOKENING MORPHLOGY **************')

# BasicTokenizer.tokenize()
print('\nOrigin Text:  ', text)
# text = convert_to_unicode(text)
text = _clean_text(text)
print('\nCleaned Text: ', text)
# fn: whitespace_tokenize()
orig_tokens = whitespace_tokenize(text)
print('\nOrig. Tokens: ', orig_tokens, end='\n\n')
Basic_split_tokens = []
for token in orig_tokens:
    if (do_lower_case) and (token not in never_split):
#         token = token.lower()
        # fn: _run_strip_accents
        t = unicodedata.normalize("NFD", token)
        # https://gist.github.com/Pusnow/aa865fa21f9557fa58d691a8b79f8a6d
        # 모든 음절을 정준 분해(Canonical Decomposition)시킴
        # '각'을 'ㄱ+ㅏ+ㄱ'으로 저장(출력되는 값은 동일)
        output = []
        for char in t:
            cat = unicodedata.category(char)
            if cat == "Mn":
                # unicode category가 "Mark, Nonspacing"일 경우 pass
                continue
            output.append(char)
        token = "".join(output)
        print('\tstripped accent+norm(NFD) Token : '+t)
    # fn: _run_split_on_punc()
    if text in never_split:
        token = [text]
    else:
        chars = list(token)
        i, start_new_word = 0, True
        output = []
        print('\tchars : '+str(chars))
        while i < len(chars):
            char = chars[i]
            if _is_punctuation(char):
                # 공백이면 [" "]를 추가하고 새로운 단어로 시작
                output.append([char])
                start_new_word = True
            else:
                # 공백이 아닐 경우,
                if start_new_word:
                    # 새로운 문자로 시작할 경우 빈 리스트 추가.
                    output.append([])
                # 해당 단어부터 시작하도록 start_new_word는 False로 setting.
                start_new_word = False
                # 위에 추가한 빈 리스트에 각각 character를 채워넣음
                output[-1].append(char)
            i += 1
        token = ["".join(x) for x in output]
    Basic_split_tokens.extend(token)
print('\nBasic_split_tokens : ', Basic_split_tokens)
Basic_output_tokens = whitespace_tokenize((' '.join(Basic_split_tokens)).strip())
print('\nBasic_output Tokens: ', Basic_output_tokens, end='\n\n')

Total_split_tokens = [] # 최종 tokenize 결과 저장
print('************** START GREEDY LONGEST MATCH FIRST ALGORITHM **************')
for tokens in Basic_output_tokens:
    tokens += '_' # adding '_'
    # WordpieceTokenizer.tokenize()
    unk_token = "[UNK]"
    max_input_chars_per_word = 100
    # greedy longest-match-first algorithm to perform tokenization
    # using the given vocabulary
    tokens = convert_to_unicode(tokens)
    WP_output_tokens = []
    # fn: whitespace_tokenize
    tokens = whitespace_tokenize(tokens)
    # start lmf algorithm!
    print('\t', tokens)
    for token in tokens:
        chars = list(token)
        if len(chars) > max_input_chars_per_word: # 100
            # max word로 설정한 글자 수를 넘길 경우, UNK 처리
            WP_output_tokens.append(unk_token)
            continue
        is_bad = False
        start = 0
        sub_tokens = []
        while start < len(chars):
            end = len(chars)
            cur_substr = None
            # 첫번째 글자부터 천천히 vocab에 있는 단어인지 체크
            while start < end:
                substr = "".join(chars[start:end])
                # do_lower_case == True일 경우에
                # 위에서 Canonical Decomposition 과정을 거쳤기 때문에
                # 이를 다시 Composition해줘야 vocab의 단어와 비교 가능하다.
                substr = unicodedata.normalize('NFC', substr)
                print('\t\t\t'+substr, end='')
                if substr in vocab:
                    # 만약 해당 단어가 vocab에 있다면 해당 단어로 break
                    cur_substr = substr
                    print('\r\t\t\t{:<15}{}'.format(
                        cur_substr, '--|> Exists in vocab_file.'))
                    break
                end -= 1
                print()
            # 만일 못찾았을 경우, (1)로 가서 [UNK] 처리.
            if cur_substr is None:
                is_bad = True
                break
            sub_tokens.append(cur_substr)
            # 어미를 추가하기 위해 start에 end값을 할당
            start = end
        if is_bad: # --- (1)
            WP_output_tokens.append(unk_token)
        else:
            # 정상적으로 끝났다면 sub_token을 결과값에 할당
            WP_output_tokens.extend(sub_tokens)
    for sub_token in WP_output_tokens:
        Total_split_tokens.append(sub_token)
print('\nTotal_split_tokens : ', Total_split_tokens)

************** START TOKENING MORPHLOGY **************

Origin Text:   XXX/SL 고객/NNG 님/XSN 항상/MAG XXX/SL 은행/NNG 모란/NNG 역/NNG 지점/NNG 을/JKO 이용/NNG 하/XSV 어/EC 주/VX 시/EP 는/ETM 고객/NNG 님/XSN 께/JKB 감사/NNG 의/JKG 마음/NNG 을/JKO 전하/VV ㅂ니다/EF ./SF 혹시/MAG 업무/NNG 와/JKB 관련/NNG 하/XSV 어/EC 궁금하/VA ㄴ/ETM 점/NNG 이/JKS 있/VA 으시/EP 면/EC 이/MM 번호/NNG 로/JKB 연락/NNG 주/VV 시/EP 기/ETN 바라/VV ㅂ니다/EF ./SF 성심껏/MAG 돕/VV 아/EC 드리/VX 겠/EP 습니다/EF ./SF 또/MAG 혹시/MAG 고객/NNG 만족/NNG 도/NNG 조사/NNG 전화/NNG 를/JKO 받/VV 으시/EP 면/EC 매우/MAG 동의/NNG 하/XSV ㄴ다/EF 로/JKB 칭찬/NNG 하/XSV 어/EC 주/VX 시/EP 어요/EF 조금/NNG 은/JX 쌀쌀하/VA ㄴ/ETM 10/SN 월/NNB 의/JKG 첫주/NNG 이/VCP ㅂ니다/EF ./SF 환절/NNG 기/XSN 감기/NNG 조심/NNG 하/XSV 시/EP 고/EC 따듯하/VA ㄴ/ETM 차/NNG 와/JC 함께/MAG 건강/NNG 하/XSA ㄴ/ETM 한/MM 주/NNB 보내/VV 시/EP 기/ETN 바라/VV ㅂ니다/EF ./SF XXX/SL 은행/NNG 모란/NNG 역/NNG XXX/SL 올림/NNG 

Cleaned Text:  XXX/SL 고객/NNG 님/XSN 항상/MAG XXX/SL 은행/NNG 모란/NNG 역/NNG 지점/NNG 을/JKO 이용/NNG 하/XSV 어/EC 주/VX 시/EP 는/ETM 고객/NNG 님/XSN 께/JKB 감사/NNG 의/JKG 마음/NNG 을/JKO 전하/VV ㅂ니다/EF ./SF 혹시/MAG 업무/NNG 와/JKB 관련

단어 사전에 있는대로 짤라버린다.

In [178]:
tokens_a = Total_split_tokens 

In [181]:
print(tokens_a)
if len(tokens_a) > max_seq_length - 2:
    print('cutting')
    tokens_a = tokens_a[:max_seq_length-2]
print(tokens_a)

['X', 'X', 'X/SL_', '고객/NNG_', '님/XSN_', '항상/MAG_', 'X', 'X', 'X/SL_', '은행/NNG_', '모', '란/NNG_', '역/NNG_', '지점/NNG_', '을/JKO_', '이용/NNG_', '하/XSV_', '어/EC_', '주/VX_', '시/EP_', '는/ETM_', '고객/NNG_', '님/XSN_', '께/JKB_', '감사/NNG_', '의/JKG_', '마음/NNG_', '을/JKO_', '전하/VV_', 'ㅂ니다/EF_', './SF_', '혹시/MAG_', '업무/NNG_', '와/JKB_', '관련/NNG_', '하/XSV_', '어/EC_', '궁금하/VA_', 'ㄴ/ETM_', '점/NNG_', '이/JKS_', '있/VA_', '으시/EP_', '면/EC_', '이/MM_', '번호/NNG_', '로/JKB_', '연락/NNG_', '주/VV_', '시/EP_', '기/ETN_', '바라/VV_', 'ㅂ니다/EF_', './SF_', '성', '심', '껏/MAG_', '돕/VV_', '아/EC_', '드리/VX_', '겠/EP_', '습니다/EF_', './SF_', '또/MAG_', '혹시/MAG_', '고객/NNG_', '만족/NNG_', '도/NNG_', '조사/NNG_', '전화/NNG_', '를/JKO_', '받/VV_', '으시/EP_', '면/EC_', '매우/MAG_', '동의/NNG_', '하/XSV_', 'ㄴ다/EF_', '로/JKB_', '칭찬/NNG_', '하/XSV_', '어/EC_', '주/VX_', '시/EP_', '어요/EF_', '조금/NNG_', '은/JX_', '쌀', '쌀', '하/VA_', 'ㄴ/ETM_', '10/SN_', '월/NNB_', '의/JKG_', '첫', '주/NNG_', '이/VCP_', 'ㅂ니다/EF_', './SF_', '환', '절/NNG_', '기/XSN_', '감기/NNG_', '조심/NNG_', '하/XSV_', 

In [203]:
tokens = []
segment_ids = []
tokens.append("[CLS]")
segment_ids.append(0)
for token in tokens_a:
    tokens.append(token)
    segment_ids.append(0)
tokens.append("[SEP]")
segment_ids.append(0)

In [204]:
print(segment_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [206]:
print(tokens)

['[CLS]', 'X', 'X', 'X/SL_', '고객/NNG_', '님/XSN_', '항상/MAG_', 'X', 'X', 'X/SL_', '은행/NNG_', '모', '란/NNG_', '역/NNG_', '지점/NNG_', '을/JKO_', '이용/NNG_', '하/XSV_', '어/EC_', '주/VX_', '시/EP_', '는/ETM_', '고객/NNG_', '님/XSN_', '께/JKB_', '감사/NNG_', '의/JKG_', '마음/NNG_', '을/JKO_', '전하/VV_', 'ㅂ니다/EF_', './SF_', '혹시/MAG_', '업무/NNG_', '와/JKB_', '관련/NNG_', '하/XSV_', '어/EC_', '궁금하/VA_', 'ㄴ/ETM_', '점/NNG_', '이/JKS_', '있/VA_', '으시/EP_', '면/EC_', '이/MM_', '번호/NNG_', '로/JKB_', '연락/NNG_', '주/VV_', '시/EP_', '기/ETN_', '바라/VV_', 'ㅂ니다/EF_', './SF_', '성', '심', '껏/MAG_', '돕/VV_', '아/EC_', '드리/VX_', '겠/EP_', '습니다/EF_', './SF_', '또/MAG_', '혹시/MAG_', '고객/NNG_', '만족/NNG_', '도/NNG_', '조사/NNG_', '전화/NNG_', '를/JKO_', '받/VV_', '으시/EP_', '면/EC_', '매우/MAG_', '동의/NNG_', '하/XSV_', 'ㄴ다/EF_', '로/JKB_', '칭찬/NNG_', '하/XSV_', '어/EC_', '주/VX_', '시/EP_', '어요/EF_', '조금/NNG_', '은/JX_', '쌀', '쌀', '하/VA_', 'ㄴ/ETM_', '10/SN_', '월/NNB_', '의/JKG_', '첫', '주/NNG_', '이/VCP_', 'ㅂ니다/EF_', './SF_', '환', '절/NNG_', '기/XSN_', '감기/NNG_', '조심/NNG_', '

In [207]:
# convert_tokens_to_ids
def convert_by_vocab(vocab, items):
    output = []
    for item in items:
        output.append(vocab[item])
    return output

input_ids = convert_by_vocab(vocab, tokens)
print(input_ids)

[2, 3047, 3047, 1496, 1291, 1123, 2547, 3047, 3047, 1496, 994, 315, 1692, 375, 3277, 11, 456, 9, 20, 129, 388, 22, 1291, 1123, 3353, 1308, 13, 588, 11, 276, 158, 7, 5865, 1579, 101, 266, 9, 20, 4511, 10, 187, 16, 38, 4506, 71, 80, 1883, 31, 2597, 359, 388, 49, 2019, 158, 7, 270, 855, 5181, 2544, 62, 4971, 124, 116, 7, 179, 5865, 1291, 2379, 356, 268, 823, 19, 78, 4506, 71, 1210, 2680, 9, 41, 31, 3998, 9, 20, 129, 388, 526, 4380, 21, 9212, 9212, 248, 10, 113, 60, 13, 4508, 211, 15, 158, 7, 1807, 1509, 284, 9869, 4315, 9, 388, 23, 2577, 9115, 10, 208, 56, 162, 1394, 42, 10, 92, 2227, 561, 388, 49, 2019, 158, 7, 3047, 3047, 3]


In [208]:
# The mask has 1 for real tokens and 0 for padding tokens. Only real
# tokens are attended to.
input_mask = [1] * len(input_ids)
print(input_mask)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [209]:
# Zero-pad up to the sequence length.
while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

In [210]:
assert len(input_ids) == max_seq_length
assert len(input_mask) == max_seq_length
assert len(segment_ids) == max_seq_length

In [213]:
label_id = label_map[example.label]
label_id

0

In [216]:
tf.logging.info('*** Example ***')
tf.logging.info('guid: %s' % (example.guid))
tf.logging.info('tokens" %s' % " ".join(
    [printable_text(x) for x in tokens]))
tf.logging.info('input_ids: %s' % " ".join([str(x) for x in input_ids]))
tf.logging.info('input_mask: %s' % " ".join([str(x) for x in input_mask]))
tf.logging.info('segment_ids: %s' % " ".join([str(x) for x in segment_ids]))
tf.logging.info('label: %s (id = %d)' % (example.label, label_id))

INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train-6
INFO:tensorflow:tokens" [CLS] X X X/SL_ 고객/NNG_ 님/XSN_ 항상/MAG_ X X X/SL_ 은행/NNG_ 모 란/NNG_ 역/NNG_ 지점/NNG_ 을/JKO_ 이용/NNG_ 하/XSV_ 어/EC_ 주/VX_ 시/EP_ 는/ETM_ 고객/NNG_ 님/XSN_ 께/JKB_ 감사/NNG_ 의/JKG_ 마음/NNG_ 을/JKO_ 전하/VV_ ㅂ니다/EF_ ./SF_ 혹시/MAG_ 업무/NNG_ 와/JKB_ 관련/NNG_ 하/XSV_ 어/EC_ 궁금하/VA_ ㄴ/ETM_ 점/NNG_ 이/JKS_ 있/VA_ 으시/EP_ 면/EC_ 이/MM_ 번호/NNG_ 로/JKB_ 연락/NNG_ 주/VV_ 시/EP_ 기/ETN_ 바라/VV_ ㅂ니다/EF_ ./SF_ 성 심 껏/MAG_ 돕/VV_ 아/EC_ 드리/VX_ 겠/EP_ 습니다/EF_ ./SF_ 또/MAG_ 혹시/MAG_ 고객/NNG_ 만족/NNG_ 도/NNG_ 조사/NNG_ 전화/NNG_ 를/JKO_ 받/VV_ 으시/EP_ 면/EC_ 매우/MAG_ 동의/NNG_ 하/XSV_ ㄴ다/EF_ 로/JKB_ 칭찬/NNG_ 하/XSV_ 어/EC_ 주/VX_ 시/EP_ 어요/EF_ 조금/NNG_ 은/JX_ 쌀 쌀 하/VA_ ㄴ/ETM_ 10/SN_ 월/NNB_ 의/JKG_ 첫 주/NNG_ 이/VCP_ ㅂ니다/EF_ ./SF_ 환 절/NNG_ 기/XSN_ 감기/NNG_ 조심/NNG_ 하/XSV_ 시/EP_ 고/EC_ 따 듯하/VA_ ㄴ/ETM_ 차/NNG_ 와/JC_ 함께/MAG_ 건강/NNG_ 하/XSA_ ㄴ/ETM_ 한/MM_ 주/NNB_ 보내/VV_ 시/EP_ 기/ETN_ 바라/VV_ ㅂ니다/EF_ ./SF_ X X [SEP]
INFO:tensorflow:input_ids: 2 3047 3047 1496 1291 1123 2547 3047 3047 1496 994 315 1692 37

In [217]:
feature = InputFeatures(
    input_ids=input_ids,
    input_mask=input_mask,
    segment_ids=segment_ids,
    label_id=label_id,
    is_real_example=True)

In [223]:
def create_int_feature(values):
    f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
    return f

In [227]:
features = collections.OrderedDict()

features['input_ids'] = create_int_feature(feature.input_ids)
features['input_mask'] = create_int_feature(feature.input_mask)
features['input_ids'] = create_int_feature(feature.input_ids)
features['segment_ids'] = create_int_feature(feature.segment_ids)
features['label_ids'] = create_int_feature([feature.label_id])
features['is_real_example'] = create_int_feature([int(feature.is_real_example)])

print(features)

OrderedDict([('input_ids', int64_list {
  value: 2
  value: 3047
  value: 3047
  value: 1496
  value: 1291
  value: 1123
  value: 2547
  value: 3047
  value: 3047
  value: 1496
  value: 994
  value: 315
  value: 1692
  value: 375
  value: 3277
  value: 11
  value: 456
  value: 9
  value: 20
  value: 129
  value: 388
  value: 22
  value: 1291
  value: 1123
  value: 3353
  value: 1308
  value: 13
  value: 588
  value: 11
  value: 276
  value: 158
  value: 7
  value: 5865
  value: 1579
  value: 101
  value: 266
  value: 9
  value: 20
  value: 4511
  value: 10
  value: 187
  value: 16
  value: 38
  value: 4506
  value: 71
  value: 80
  value: 1883
  value: 31
  value: 2597
  value: 359
  value: 388
  value: 49
  value: 2019
  value: 158
  value: 7
  value: 270
  value: 855
  value: 5181
  value: 2544
  value: 62
  value: 4971
  value: 124
  value: 116
  value: 7
  value: 179
  value: 5865
  value: 1291
  value: 2379
  value: 356
  value: 268
  value: 823
  value: 19
  value: 78
  value: 45

In [228]:
tf_example = tf.train.Example(features=tf.train.Features(feature=features))

In [229]:
tf_example

features {
  feature {
    key: "input_ids"
    value {
      int64_list {
        value: 2
        value: 3047
        value: 3047
        value: 1496
        value: 1291
        value: 1123
        value: 2547
        value: 3047
        value: 3047
        value: 1496
        value: 994
        value: 315
        value: 1692
        value: 375
        value: 3277
        value: 11
        value: 456
        value: 9
        value: 20
        value: 129
        value: 388
        value: 22
        value: 1291
        value: 1123
        value: 3353
        value: 1308
        value: 13
        value: 588
        value: 11
        value: 276
        value: 158
        value: 7
        value: 5865
        value: 1579
        value: 101
        value: 266
        value: 9
        value: 20
        value: 4511
        value: 10
        value: 187
        value: 16
        value: 38
        value: 4506
        value: 71
        value: 80
        value: 1883
        value: 31
        value

In [230]:
writer.write(tf_example.SerializeToString())

In [231]:
writer.close()

In [232]:
train_file

'./output_dir/smishing/train.tf_record'

In [233]:
tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", FLAGS.train_batch_size)
tf.logging.info("  Num steps = %d", num_train_steps)

INFO:tensorflow:***** Running training *****
INFO:tensorflow:  Num examples = 100
INFO:tensorflow:  Batch size = 32
INFO:tensorflow:  Num steps = 9


# Spacing안한 전체 데이터로 돌려보기!

1. 데이터 준비

In [236]:
processor = SmishingProcessor()
label_list = processor.get_labels()

# get train samples
train_examples = processor.get_train_examples(dacon_path, 'train.tsv')
num_train_steps = int(
    len(train_examples) / FLAGS.train_batch_size * FLAGS.num_train_epochs)
num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)

In [237]:
# record ETRI model weights
FLAGS.init_checkpoint = path + 'model.ckpt'

model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=len(label_list),             # 2
    init_checkpoint=FLAGS.init_checkpoint,  # None
    learning_rate=FLAGS.learning_rate,      # 5e-05
    num_train_steps=num_train_steps,        # 22195
    num_warmup_steps=num_warmup_steps,      # 2219
    use_tpu=FLAGS.use_tpu,                  # False
    use_one_hot_embeddings=FLAGS.use_tpu)   # False

# If TPU is not available, this will fall back to normal Estimator on CPU
# or GPU
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,                        # False
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,      # 32
    eval_batch_size=FLAGS.eval_batch_size,        # 8
    predict_batch_size=FLAGS.predict_batch_size   # 8
)

FLAGS.output_dir = './output_dir/smishing/'

tf.gfile.MakeDirs(FLAGS.output_dir)

INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\jinma\\AppData\\Local\\Temp\\tmpafvhq326', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002199E5846D8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, nu

In [240]:
def file_based_convert_examples_to_features(
    examples, label_lsit, max_seq_length, tokenizer, output_file):
    
    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example, label_list,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["label_ids"] = create_int_feature([feature.label_id])
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()

In [244]:
train_file = os.path.join(FLAGS.output_dir, 'train_non_spacing.tf_record')

In [ ]:
file_based_convert_examples_to_features